In [ ]:
%matplotlib nbagg
from sympy import *
import numpy as np
import scipy as sp
import scipy.special as spc
import matplotlib.pyplot as plt
init_printing()

# Вывод полной аппартной функции для Троицк ν-Mass

Полная аппаратная функция является свёрткой функции пропускания спектрометра:

$$
R(E,U) = \left\{
\begin{aligned}
&0& E < U\\
&1 & E > U+\Delta\\
&(E - U) / \Delta \qquad& \mbox{otherwise} \\
\end{aligned}
\right.
$$

и конечной ширины спектра электронов вылетающих из пушки являющихся нормальным. Так как конволюция коммутирует с трансляцией мы будем рассматривать случай $U=0$. Таким образом нам надо вычислить интеграл

$$
\int_{-\infty}^{\infty}dx\, R(x,0) N_{(0,\sigma)}(E-x) = 
\int_{-\infty}^{\infty}dx\, R(x,0) N_{(E,\sigma)}(x) 
$$

Интеграл распадается на два интеграла

$$
\int_{\Delta}^{\infty}dx\, N_{(E,\sigma)}(x) +
\int_{0}^{\Delta}dx\, x \cdot N_{(E,\sigma)}(x) 
$$

In [ ]:
# Введём переменные
x,y,z,U,E,A,B  = symbols('x y z U E A B', real=True)
mu     = symbols('mu',    real=True)
sigma  = symbols('sigma', real=True, positive=True)
Delta  = symbols('Delta', real=True, positive=True)

In [ ]:
def gaus(mu,sigma,x) :
    return 1 / (sqrt(2*pi) * sigma) * exp( -(x-mu)**2 / (2*sigma**2))

# Первое слагаемое

$$
\int_{\Delta}^{\infty}dx\, N_{(E,\sigma)}(x)
$$

In [ ]:
i_part1 = Integral(gaus(E,sigma,x), (x,Delta,oo))
i_part1

In [ ]:
i_part1_r = i_part1.transform(x/sigma, y).transform(y - E/sigma, z).doit()
i_part1_r

sympy туповат, досчитаем руками

In [ ]:
simplify(i_part1_r)

In [ ]:
part1 = factor(integrate( 1/ sqrt(2*pi) * exp(-x**2/2), (x,(Delta-E)/sigma,oo)))
part1

Получаем выражение:

$$
\frac{1}{2} \operatorname{erfc}(\frac{\Delta-E}{\sqrt{2}\sigma})
$$

# Второе слагаемое

$$
\int_{-\infty}^{\infty}dx\, R(x,0) N_{(E,\sigma)}(x) 
$$

In [ ]:
i_part2 = Integral( x / Delta * gaus(E,sigma,x), (x,0,Delta))
i_part2

In [ ]:
i_part2_1 = simplify( i_part2.transform(x/sigma, y).transform(y-E/sigma, z) )
i_part2_1

In [ ]:
i_part2_2 = simplify(i_part2_1.doit())
print(i_part2_2)
i_part2_2

In [ ]:
factor( E*erf(sqrt(2)*E/(2*sigma))/(2*Delta) + E*erf(sqrt(2)*Delta/(2*sigma) - sqrt(2)*E/(2*sigma))/(2*Delta) )

In [ ]:
r = factor( sqrt(2)*sigma*exp(-E**2/(2*sigma**2))/(2*sqrt(pi)*Delta) - sqrt(2)*sigma*exp(-(Delta - E)**2/(2*sigma**2))/(2*sqrt(pi)*Delta) )
print(r)
r

In [ ]:
simplify( sqrt(2)*sigma*exp(-Delta**2/(2*sigma**2))*exp(-E**2/(2*sigma**2))*exp(Delta*E/sigma**2)/(2*sqrt(pi)*Delta) )

Итого мы получаем сумму следующих выражений:

$$
\frac{E}{2\Delta}\left[
   \operatorname{erf}\left(\frac{E}{\sqrt{2}\sigma}\right)
 + \operatorname{erf}\left(\frac{\Delta-E}{\sqrt{2}\sigma}\right)
\right]
$$

и

$$
\frac{\sigma}{\sqrt{2\pi}\Delta}\exp\left(-\frac{(E-\Delta)^2}{2\sigma^2}\right)
\left[
  \exp\left(\frac{\Delta^2-\Delta E}{2\sigma^2}\right) - 1
\right]
$$

# Численная формула

Теперь надо перевести формулы в численный код на numpy.

In [ ]:
valDelta = 0.35
valSigma = 0.27179

Построим сначала графики для аналитического решения:

In [ ]:
r  = (part1 + i_part2_2).subs(Delta,valDelta).subs(sigma,valSigma)
r1 = (part1).subs(Delta,valDelta).subs(sigma,valSigma)
r2 = (i_part2_2).subs(Delta,valDelta).subs(sigma,valSigma)
r3 = (E*erf(sqrt(2)*E/(2*sigma))/(2*Delta) + E*erf(sqrt(2)*Delta/(2*sigma) - sqrt(2)*E/(2*sigma))/(2*Delta)).subs(Delta,valDelta).subs(sigma,valSigma)
r4 = (- sqrt(2)*sigma*exp(-(Delta - E)**2/(2*sigma**2))/(2*sqrt(pi)*Delta) + sqrt(2)*sigma*exp(-E**2/(2*sigma**2))/(2*sqrt(pi)*Delta)).subs(Delta,valDelta).subs(sigma,valSigma)
#
xs = np.linspace(-1,1,100)
ys = [N(r.subs(E,x_)) for x_ in xs]
#
plt.figure()
plt.grid()
plt.plot(xs,ys)
plt.plot(xs, [N(r1.subs(E,x_)) for x_ in xs])
plt.plot(xs, [N(r2.subs(E,x_)) for x_ in xs] )
plt.plot(xs, [N(r3.subs(E,x_)) for x_ in xs] )
plt.plot(xs, [N(r4.subs(E,x_)) for x_ in xs] )
None

И для численного

In [ ]:
def fun_part1(delta,sig,e) :
    denom = np.sqrt(2) * sig
    return spc.erfc((delta-e) / denom) / 2
def fun_part2(delta,sig,e) :
    denom = np.sqrt(2) * sig
    return e / (2*delta) * (spc.erf(e/denom) + spc.erf((delta-e)/denom))
def fun_part3(delta,sig,e) :
    denom2 = 2 * sig**2
    return ( sig/(np.sqrt(2*np.pi)*delta) 
           * (np.exp( -e**2 / denom2) - np.exp( -(e-delta)**2 / denom2 ))
           )

In [ ]:
xs = np.linspace(-1,1,100)
valDelta = 0.35
valSigma = 0.27179
#
plt.figure()
plt.grid()
plt.plot([xs[0],0,valDelta,xs[-1]], [0,0,1,1])
plt.plot(xs, fun_part1(valDelta,valSigma,xs))
plt.plot(xs, fun_part2(valDelta,valSigma,xs))
plt.plot(xs, fun_part3(valDelta,valSigma,xs))
plt.plot(xs, fun_part1(valDelta,valSigma,xs)+fun_part2(valDelta,valSigma,xs)+fun_part3(valDelta,valSigma,xs))
plt.plot(xs, fun_part2(valDelta,valSigma,xs)+fun_part3(valDelta,valSigma,xs))
None